# Automatic Mask Generation with SAM 2

[![image](https://studiolab.sagemaker.aws/studiolab.svg)](https://studiolab.sagemaker.aws/import/github/opengeos/segment-geospatial/blob/main/docs/examples/sam2_automatic.ipynb)
[![image](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/opengeos/segment-geospatial/blob/main/docs/examples/sam2_automatic.ipynb)

This notebook shows how to segment objects from an image using the Segment Anything Model 2 (SAM2) with a few lines of code.

Make sure you use GPU runtime for this notebook. For Google Colab, go to `Runtime` -> `Change runtime type` and select `GPU` as the hardware accelerator.

## Install dependencies

Uncomment and run the following cell to install the required dependencies.

In [13]:
%pip install -U segment-geospatial[samgeo2]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 136.4/136.4 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.4/88.4 kB 7.1 MB/s eta 0:00:00


In [18]:
!pip install leafmap
import leafmap
from samgeo import SamGeo2

## Create an interactive map

In [16]:
m = leafmap.Map(center=[-4.336283776655342, 15.2282495810964], zoom=19)
m.add_basemap("SATELLITE")
m

Map(center=[-4.336283776655342, 15.2282495810964], controls=(ZoomControl(options=['position', 'zoom_in_text', …


Pan and zoom the map to select the area of interest. Use the draw tools to draw a polygon or rectangle on the map

In [11]:
if m.user_roi_bounds() is not None:
    bbox = m.user_roi_bounds()
else:
    bbox = [-95.3704, 29.6762, -95.368, 29.6775]

## Download a sample image

In [14]:
image = "satellite.tif"
# Ensure bbox is defined if the user has not drawn an ROI
if 'm' in locals() and m.user_roi_bounds() is not None:
    bbox = m.user_roi_bounds()
else:
    bbox = [-95.3704, 29.6762, -95.368, 29.6775]

leafmap.map_tiles_to_geotiff(
    output=image, bbox=bbox, zoom=20, source="Satellite", overwrite=True
)

Downloaded image 1/40
Downloaded image 2/40
Downloaded image 3/40
Downloaded image 4/40
Downloaded image 5/40
Downloaded image 6/40
Downloaded image 7/40
Downloaded image 8/40
Downloaded image 9/40
Downloaded image 10/40
Downloaded image 11/40
Downloaded image 12/40
Downloaded image 13/40
Downloaded image 14/40
Downloaded image 15/40
Downloaded image 16/40
Downloaded image 17/40
Downloaded image 18/40
Downloaded image 19/40
Downloaded image 20/40
Downloaded image 21/40
Downloaded image 22/40
Downloaded image 23/40
Downloaded image 24/40
Downloaded image 25/40
Downloaded image 26/40
Downloaded image 27/40
Downloaded image 28/40
Downloaded image 29/40
Downloaded image 30/40
Downloaded image 31/40
Downloaded image 32/40
Downloaded image 33/40
Downloaded image 34/40
Downloaded image 35/40
Downloaded image 36/40
Downloaded image 37/40
Downloaded image 38/40
Downloaded image 39/40
Downloaded image 40/40
Saving GeoTIFF. Please wait...
Image saved to satellite.tif


You can also use your own image. Uncomment and run the following cell to use your own image.

In [ ]:
# image = '/path/to/your/own/image.tif'

Display the downloaded image on the map.

In [20]:
!pip install localtileserver
m.layers[-1].visible = False
m.add_raster(image, layer_name="Image")
m

Map(bottom=27756929.0, center=[29.67685, -95.3692], controls=(ZoomControl(options=['position', 'zoom_in_text',…

## Initialize SAM class

In [21]:
sam2 = SamGeo2(model_id="sam2-hiera-large", automatic=True)

ImportError: To use SamGeo 2, install it as:
	pip install segment-geospatial[samgeo2]

## Automatic mask generation

Segment the image and save the results to a GeoTIFF file. Set `unique=True` to assign a unique ID to each object.

In [4]:
sam2.generate(image)

NameError: name 'sam2' is not defined

In [ ]:
sam2.save_masks(output="masks.tif")

In [ ]:
sam2.show_masks(cmap="binary_r")

In [ ]:
sam2.show_masks(cmap="jet")

Show the object annotations (objects with random color) on the map.

In [ ]:
sam2.show_anns(axis="off", alpha=0.7, output="annotations.tif")

Compare images with a slider.

In [ ]:
leafmap.image_comparison(
    "satellite.tif",
    "annotations.tif",
    label1="Satellite Image",
    label2="Image Segmentation",
)

Add image to the map.

In [ ]:
m.add_raster("masks.tif", colormap="jet", layer_name="Masks", nodata=0, opacity=0.7)
m

Convert the object annotations to vector format, such as GeoPackage, Shapefile, or GeoJSON.

In [ ]:
sam2.raster_to_vector("masks.tif", "masks.gpkg")

In [ ]:
m.add_vector("masks.gpkg", layer_name="Objects")

## Automatic mask generation options

There are several tunable parameters in automatic mask generation that control how densely points are sampled and what the thresholds are for removing low quality or duplicate masks. Additionally, generation can be automatically run on crops of the image to get improved performance on smaller objects, and post-processing can remove stray pixels and holes. Here is an example configuration that samples more masks:

In [ ]:
sam2 = SamGeo2(
    model_id="sam2-hiera-large",
    apply_postprocessing=False,
    points_per_side=32,
    points_per_batch=64,
    pred_iou_thresh=0.7,
    stability_score_thresh=0.92,
    stability_score_offset=0.7,
    crop_n_layers=1,
    box_nms_thresh=0.7,
    crop_n_points_downscale_factor=2,
    min_mask_region_area=25.0,
    use_m2m=True,
)

In [ ]:
sam2.generate(image, output="masks2.tif")

In [ ]:
sam2.show_masks(cmap="jet")

In [ ]:
sam2.show_anns(axis="off", alpha=0.7, output="annotations2.tif")

Compare images with a slider.

In [ ]:
leafmap.image_comparison(
    image,
    "annotations2.tif",
    label1="Image",
    label2="Image Segmentation",
)